In [1]:
import time
from selenium import webdriver

driver = webdriver.Chrome("/Users/donghunbyun/Desktop/workspace/jupyter/bigdata/chromedriver")

In [2]:
from bs4 import BeautifulSoup

In [3]:
driver.get("https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=1&sido=&gugun=&store=")

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
    
stores = soup.select('#contents > div.content > fieldset > fieldset > div.tableType01 > table > tbody > tr')

In [4]:
store_location = stores[0].select('td')[0].text
store_name = stores[0].select('td > a')[0].text
store_address = stores[0].select('td > a')[1].text
store_phone = stores[0].select('td')[-1].text

In [5]:
print(store_location, store_name, store_address, store_phone)

충북 청주시 서원구 청주성화점 충청북도 청주시 서원구 신성화로 39 (성화동) 1~2층 043-233-6296


In [6]:
def hollys_searching(page):
    url = f"https://www.hollys.co.kr/store/korea/korStore2.do?pageNo={page}&sido=&gugun=&store="
    return url

In [7]:
import json
import requests

In [8]:
kakao_API_key = "5e96943c943636b37f24cd5c657d92b3"

In [21]:
phone = '02-403-0338'

url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + phone
headers = {"Authorization": f"KakaoAK {kakao_API_key}"}

result = json.loads(str(requests.get(url, headers=headers).text))

result
# print(float(result['documents'][0]['y']), float(result['documents'][0]['x']))

{'documents': [],
 'meta': {'is_end': True,
  'pageable_count': 0,
  'same_name': {'keyword': '02-403-0338', 'region': [], 'selected_region': ''},
  'total_count': 0}}

In [13]:
def getLatLng(phone):
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query=' + phone
    headers = {"Authorization": f"KakaoAK {kakao_API_key}"}

    result = json.loads(str(requests.get(url, headers=headers).text))
    
    print(result)
    
    location = result['documents'][0]
    
    return float(location['y']), float(location['x'])

In [14]:
def get_store(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    stores = soup.select('#contents > div.content > fieldset > fieldset > div.tableType01 > table > tbody > tr')
    
    data = []
    
    for store in stores:
        store_location = store.select('td')[0].text
        store_name = store.select('td > a')[0].text
        store_address = store.select('td > a')[1].text
        store_phone = store.select('td')[-1].text
        
        store_lat, store_lng = getLatLng(store_phone)
                
        data.append([store_location, store_name, store_address, store_phone, store_lat, store_lng])
    
    return data

In [15]:
result = []

for i in range(1, 57):
    url = hollys_searching(i)

    driver.get(url)
    
    data = get_store(driver)
    
    result += data

{'documents': [{'address_name': '충북 청주시 서원구 성화동 469', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페 > 커피전문점 > 할리스', 'distance': '', 'id': '21907652', 'phone': '043-233-6296', 'place_name': '할리스 청주성화점', 'place_url': 'http://place.map.kakao.com/21907652', 'road_address_name': '충북 청주시 서원구 신성화로 39', 'x': '127.458144521012', 'y': '36.6184379628382'}], 'meta': {'is_end': True, 'pageable_count': 1, 'same_name': {'keyword': '043-233-6296', 'region': [], 'selected_region': ''}, 'total_count': 1}}
{'documents': [{'address_name': '경기 안성시 서운면 신능리 111-23', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페 > 커피전문점 > 할리스', 'distance': '', 'id': '85167537', 'phone': '031-674-4406', 'place_name': '할리스 안성맞춤휴게소음성방향점', 'place_url': 'http://place.map.kakao.com/85167537', 'road_address_name': '경기 안성시 서운면 평택제천고속도로 40', 'x': '127.27425049714945', 'y': '36.968148409972436'}, {'address_name': '경기 안성시 서운면 신능리 111-23', 'category_group_cod

IndexError: list index out of range

In [ ]:
len(result)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(result)
df.columns = ["지역", "매장명", "주소", '위도', '경도']

In [ ]:
df.head()

In [ ]:
df.to_excel('./files/hollys_stores.xlsx')